In [6]:
%%writefile mapper_find_count.py
#!/usr/bin/env python2
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

def isWordName(word):
    if len(word) == 0 or word[0].isdigit() or (not word[0].isupper()):
        return False
    else:
        if len(word) >= 2:
            for c in word[1:]:
                if c.isupper():
                    return False
            return True
        else:
            return False

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if isWordName(word):
            print "%s\t%s\t%d" % (word.lower(), 'n', 1)
        else:
            print "%s\t%s\t%d" % (word.lower(), 'o', 1)

Overwriting mapper_find_count.py


In [2]:
%%writefile reducer_find_count.py
#!/usr/bin/env python2
import sys

current_key = None
word_sum_n = 0
word_sum_o = 0
isInit = True

for line in sys.stdin:
    try:
        key, typ, count = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue
    
    if isInit:
        if typ == 'n':
            word_sum_n += count
        elif typ == 'o':
            word_sum_o += count
        
        current_key = key
        isInit = False
        continue
    
    if current_key != key:
        if current_key:
            if float(word_sum_o)/(word_sum_n + word_sum_o) < 0.005:
                current_key = current_key[0].upper() + current_key[1:]
                print "%s\t%d" % (current_key, word_sum_n)
        word_sum_n = 0
        word_sum_o = 0
        current_key = key
    
    if typ == 'n':
        word_sum_n += count
    elif typ == 'o':
        word_sum_o += count
    
if current_key:
    if float(word_sum_o)/(word_sum_n + word_sum_o) < 0.005:
        current_key = current_key[0].upper() + current_key[1:]
        print "%s\t%d" % (current_key, word_sum_n)

Writing reducer_find_count.py


In [3]:
%%writefile reducer_getresult.py
#!/usr/bin/env python2
import sys

counter = 0

for line in sys.stdin:
    counter += 1
    if counter == 5:
        print "%s" % (line.lower())

Writing reducer_getresult.py


In [7]:
%%bash
NAMES_DIR="names"
RESULT_DIR="result"

hdfs dfs -rm -r -skipTrash ${NAMES_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${RESULT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Names_count" \
    -D mapreduce.job.reduces=2 \
    -files mapper_find_count.py,reducer_find_count.py \
    -mapper "python mapper_find_count.py" \
    -reducer "python reducer_find_count.py" \
    -input /data/wiki/en_articles_part \
    -output ${NAMES_DIR} > /dev/null
    
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Result" \
    -D mapreduce.job.reduces=1 \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2nr" \
    -files reducer_getresult.py \
    -mapper "cat" \
    -reducer "python reducer_getresult.py" \
    -input ${NAMES_DIR}/part-* \
    -output ${RESULT_DIR} > /dev/null
    
hdfs dfs -cat ${RESULT_DIR}/part-* | cat

french	5742	
	


17/11/25 08:22:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/25 08:22:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/25 08:22:12 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/25 08:22:12 INFO mapreduce.JobSubmitter: number of splits:2
17/11/25 08:22:12 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
17/11/25 08:22:12 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1511597534846_0004
17/11/25 08:22:13 INFO impl.YarnClientImpl: Submitted application application_1511597534846_0004
17/11/25 08:22:13 INFO mapreduce.Job: The url to track the job: http://1502185b6101:8088/proxy/application_1511597534846_0004/
17/11/25 08:22:13 INFO mapreduce.Job: Running job: job_1511597534846_0004
17/11/25 08:22:19 INFO mapreduce.Job: Job job_1511597534846_0004 running in uber mode : false
17/11/25 08:22:19 INFO mapreduce.Job:  map 0% reduce 0%
17/11/25 08:22:37 INFO m